# Question 1

## Just copy past the code in pyspark,make sure to include the correct path

## answer 1.a 

In [ ]:
'''calculate total number COMPLETE, CLOSED and PROCESSING transactions for  each state 
where the each of the transaction status count is greater then 50 and 
sort the final output descending order of the state and ascending order of the count.
The output should be " , " separated text file with gzip compression written 
to "/user/cloudera/answer/question1/text" and must contain single file in the destination'''

orders="/user/pruthviraj/sqoop_text/orders"
orders=sqlContext.read.format("text").load(orders)
orders=orders.selectExpr("cast(split(value,',') [0] as int) order_customer_id",
"cast(split(value,',') [1] as date) order_date",
"cast(split(value,',') [2] as int) order_id",
"cast(split(value,',') [3] as string) order_status")

customer="file:///D://data-master/retail_db/customers"
customer=spark.read.format("text").load(customer)
customer=customer.selectExpr("cast(split(value,',') [0] as int) customer_id",
"cast(split(value,',') [1] as string) customer_fname",
"cast(split(value,',') [2] as string) customer_lname",
"cast(split(value,',') [3] as string) customer_email",
"cast(split(value,',') [4] as string) customer_password",
"cast(split(value,',') [5] as string) customer_street",
"cast(split(value,',') [6] as string) customer_city",
"cast(split(value,',') [7] as string) customer_state",
"cast(split(value,',') [8] as string) customer_zipcode")

from pyspark.sql import functions as f

customer.join(orders,customer.customer_id==orders.order_customer_id,"inner").\
groupBy(customer.customer_state,orders.order_status).\
agg(f.count(orders.order_customer_id).alias("count")).\
filter("order_status in ('COMPLETE','CLOSED','PROCESSING')").where("count >50").\
orderBy(["customer_state","count"],ascending=[0,1]).\
repartition(1,customer.customer_state).\
select(f.format_string("%s,%s,%d","customer_state","order_status","count").alias('ANSWER')).\
rdd.saveAsTextFile("/user/cloudera/answer/question1/text","org.apache.hadoop.io.compress.GzipCodec")

# answer 1.b

In [ ]:
'''calculate total number COMPLETE, CLOSED and PROCESSING transactions for  each state 
where the each of the transaction status count is greater then 50 and 
sort the final output descending order of the state and ascending order of the count.
The output should be JSON file with gzip compression written 
to "/user/cloudera/answer/question1/json" and must contain three file in the destination'''

orders="/user/pruthviraj/sqoop_text/orders"
orders=sqlContext.read.format("text").load(orders)
orders=orders.selectExpr("cast(split(value,',') [0] as int) order_customer_id",
"cast(split(value,',') [1] as date) order_date",
"cast(split(value,',') [2] as int) order_id",
"cast(split(value,',') [3] as string) order_status")

customer="file:///D://data-master/retail_db/customers"
customer=spark.read.format("text").load(customer)
customer=customer.selectExpr("cast(split(value,',') [0] as int) customer_id",
"cast(split(value,',') [1] as string) customer_fname",
"cast(split(value,',') [2] as string) customer_lname",
"cast(split(value,',') [3] as string) customer_email",
"cast(split(value,',') [4] as string) customer_password",
"cast(split(value,',') [5] as string) customer_street",
"cast(split(value,',') [6] as string) customer_city",
"cast(split(value,',') [7] as string) customer_state",
"cast(split(value,',') [8] as string) customer_zipcode")

from pyspark.sql import functions as f

customer.join(orders,customer.customer_id==orders.order_customer_id,"inner").\
groupBy(customer.customer_state,orders.order_status).\
agg(f.count(orders.order_customer_id).alias("count")).\
filter("order_status in ('COMPLETE','CLOSED','PROCESSING')").where("count >50").\
orderBy(["customer_state","count"],ascending=[0,1]).\
repartition(3,customer.customer_state).toJSON().\
toJSON().saveAsTextFile("/user/cloudera/answer/question1/json","org.apache.hadoop.io.compress.GzipCodec")

## sample output for both the question is same except the writting format

In [ ]:
+-----------------+
|           ANSWER|
+-----------------+
|     TX,CLOSED,78|
| TX,PROCESSING,78|
|  TX,COMPLETE,212|
|    PR,CLOSED,556|
|PR,PROCESSING,559|
| PR,COMPLETE,1566|
|   PA,COMPLETE,91|
|   OH,COMPLETE,92|
|     NY,CLOSED,88|
| NY,PROCESSING,97|
|  NY,COMPLETE,259|
|   NJ,COMPLETE,69|
|   NC,COMPLETE,52|
|   MI,COMPLETE,72|
|     IL,CLOSED,62|
| IL,PROCESSING,68|
|  IL,COMPLETE,168|
|   GA,COMPLETE,54|
|  FL,COMPLETE,134|
|    CA,CLOSED,205|
+-----------------+